In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np
import pandas as pd 
from matplotlib import pyplot as plt
from path import Path
import gc

import optuna
from sklearn.model_selection import StratifiedKFold

from scipy.special import erfinv

import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu,True)
    
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l1
from tensorflow.keras.metrics import AUC

from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras.layers import Activation, LeakyReLU
get_custom_objects().update({'leaky-relu':Activation(LeakyReLU(alpha=0.2))})

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

/kaggle/input/porto-seguro-safe-driver-prediction/sample_submission.csv
/kaggle/input/porto-seguro-safe-driver-prediction/train.csv
/kaggle/input/porto-seguro-safe-driver-prediction/test.csv


In [3]:
def gpu_cleanup(objects):
    if objects:
        del(objects)
        K.clear_session()
        gc.collect()

In [4]:
class Config:
    input_path = Path('/kaggle/input/porto-seguro-safe-driver-prediction')
    dae_batch_size = 128 
    dae_num_epoch = 50
    dae_architecture = [1500,1500,1500]
    reuse_autoencoder = False
    
    batch_size = 128
    num_epoch = 150

    units = [64,32]
    input_dropout = 0.06
    dropout = 0.08
    activation = 'selu'
    
    cv_folds = 5
    nas = False
    random_state = 0 

config = Config()

In [5]:
train = pd.read_csv(config.input_path / 'train.csv', index_col='id')
test = pd.read_csv(config.input_path / 'test.csv', index_col='id')
submission = pd.read_csv(config.input_path / 'sample_submission.csv', index_col='id')

calc_features = [feat for feat in train.columns if '_calc' in feat]
cat_features = [feat for feat in train.columns if '_cat' in feat]

target = train["target"]
train = train.drop(['target'],axis=1)

train = train.drop(calc_features, axis=1)
test = test.drop(calc_features, axis=1)

train = pd.get_dummies(train, columns = cat_features)
test = pd.get_dummies(test, columns = cat_features)

assert((train.columns==test.columns).all())

In [6]:
print("Applying GaussRank to columns: ", end='')
to_normalize = list()
for k, col in enumerate(train.columns):
    if '_bin' not in col and '_cat' not in col and '_missing' not in col:
        to_normalize.append(col)
print(to_normalize)

Applying GaussRank to columns: ['ps_ind_01', 'ps_ind_03', 'ps_ind_14', 'ps_ind_15', 'ps_reg_01', 'ps_reg_02', 'ps_reg_03', 'ps_car_11', 'ps_car_12', 'ps_car_13', 'ps_car_14', 'ps_car_15']


In [7]:
def to_gauss(x): return np.sqrt(2)* erfinv(x)

def normalize(data, norm_cols):
    n = data.shape[0]
    for col in norm_cols:
        sorted_idx = data[col].sort_values().index.tolist()
        uniform = np.linspace(start=-0.99, stop=0.99, num=n)
        normal = to_gauss(uniform)
        normalized_col = pd.Series(index = sorted_idx, data=normal)
        data[col] = normalized_col
    return data

train = normalize(train, to_normalize)
test = normalize(test, to_normalize)

In [8]:
features = train.columns
train_index = train.index
test_index = test.index

train = train.values.astype(np.float32)
test = test.values.astype(np.float32)

In [9]:
def plt_keras_history(history, measures):
    row = len(measures) // 2 + len(measures)%2
    fig, panels = plt.subplots(rows, 2, figsize=(15,5))
    plt.subplots_adjust(top = 0.99, bottom = 0.01,
                       hspace = 0.4, wspace = 0.2)
    try:
        panels = [item for sublist in panels for item in sublist]
    except:
        pass
    for k, measure in enumerate(measures):
        panel = panels[k]
        panel.set_title(measure + 'history')
        panel.plot(history.epoch, history.history[measure],
                  label='Train ' + measure)
        try:
            panel.plot(history.epoch, history.history['val_'+measure],
                      label = "Validation "+measure)
        except:
            pass
        panel.set(xlabel='epochs', ylabel=measure)
        panel.legend()
    plt.show(fig)
    

from numba import jit
@jit

def eval_gini(y_true, y_pred):
    y_true = np.asarray(y_true)
    y_true = y_true[np.argsort(y_pred)]
    ntrue = 0
    gini = 0
    delta = 0 
    n = len(y_true)
    for i in range(n-1, -1, -1):
        y_i = y_true[i]
        ntrue += y_i 
        gini += y_i * delta
        delta += 1 - y_i
    gini = 1 - 2 * gini / (ntrue *(n - ntrue))
    return gini 

/tmp/ipykernel_28/2679286598.py:28: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def eval_gini(y_true, y_pred):


In [11]:
## create a generator that provides shuffled chunks of the data based on batch size.

def batch_generator(x, batch_size, shuffle=True, random_state = None):
    batch_index = 0 
    n = x.shape[0]
    while True:
        if batch_index == 0:
            index_array = np.arange(n)
            if shuffle:
                np.random.seed(seed=random_state)
                index_array = np.random_permutation(n)
        current_index = (batch_index * batch_size)%n
        if n >= current_index + batch_size:
            current_batch_size = batch_size
            batch_index +=1
        else:
            current_batch_size = n - current_index
            batch_index=0
        batch = x[index_array[current_index: current_index + current_batch_size]]
        
        yield batch

4

In [11]:
## mixup generator that return batches of data whose values have been partially swapped
## to create some noise and augment the data to avoid the DAE overfitting to the training dataset.
## Can be viewed as a way to inject random values into the dataset and create many more examples
## to be used for trianing. 15% of random values at each batch
## Also, since the random values are picked from the same features it's not totally random
## because they are from the same distribution of the original features

def mixup_generator(X, batch_size, swaprate =0.15, shuffle = True, random_state = None):
    if random_state is None:
        random_state = np.randint(0,999)
    num_features = X.shape[1]
    num_swaps = int(num_features * swaprate)
    generator_a = batch_generator(X, batch_size, shuffle, random_state)
    generator_b = batch_generator(X, batch_size, shuffle, random_state+1)
    
    while True:
        batch = next(generator_a)
        mixed_batch = batch.copy()
        effective_batch_size = batch.shape[0]
        alternative_batch = next(generator_b)
        assert((batch != alternative_batch).any())
        for i in range(effective_batch_size):
            swap_idx = np.random.choice(num_features, num_swaps, replace=False)
            mixed_batch[i, swap_idx] = alternative_batch[i,swap_idx]
        yield (mixed_batch, batch)
    

In [16]:
### the get DAE is the function that buils the denoising autoencoder.

def get_DAE(X, architecture = [1500,1500,1500]):
    features = X.shape[1]
    inputs = Input((features,))
    for i, nodes in enumerature(architecture):
        layer = Dense(nodes, activation='relu', 
                     use_bias=False, name = f"code_{i+1}")
        if i==O:
            x = layer(inputs)
        else:
            x = layer(x)
        x = BatchNormalization()(x)
    outputs = Dense(features, activation = 'linear')(x)
    model = Model(inputs=inputs, outputs = outputs)
    model.compile(optimizer='adam', loss='mse',
                 metrics=['mse', 'mae'])
    return model

In [17]:
def extract_dae_features(autoencoder, X, layers=[3], batch_size=128):
    data=[]
    for layers in layers:
        if layer==0:
            data.append(X)
        else:
            get_layer_output = Model([autoencoder.layers[0].input],
                                    [autoencoer.layers[layer].output])
            layer_output = get_layer_output.predict(X, batch_size=batch_size)
            data.append(layer_output)
            
    data = np.hstack(data)
    return data
            

In [18]:
def autoencoder_fitting(X_train, X_valid, filename = 'dae', random_state = None,
                       suppress_output=False):
    if suppress_output:
        verbose=0
    else:
        verbose=2
        print("Fitting a denoising autoencoder")
        tf.random.set_seed(seed=random_state)
        generator = mixup_generator(X_train,
                                   batch_size=config.dae_batch_size,
                                   swaprate=0.15,
                                   random_state=config.random_state)
        dae = get_DAE(X_train, architecture=config.dae_architecture)
        steps_per_epoch = np.ceil(X_train.shape[0]/config.dae_batch_size)
        
        early_stopping = EarlyStopping(monitor='val_mse',
                                      mode='min',
                                      patience=5,
                                      restore_best_weights=True,
                                      verbose=0)
        history = dae.fit(generator,
                         steps_per_epoch = steps_per_epoch,
                         epochs = config.dae_num_epoch,
                         validation_data=(X_valid, X_valid),
                         callbakcs = [early_stopping],
                         verbose = verbose)
        if not suppress_output: plot_keras_history(history,measures=['mse','mae'])
        dae.save(filename)
        return dae